# Deploy Parcel

In [1]:
import pprint, time, sys

In [2]:
varClouderaPath = "/vagrant/notebooks/cloudera"
sys.path.append(varClouderaPath)

varClouderaPath

'/vagrant/notebooks/cloudera'

In [3]:
sys.path

['',
 '/opt/anaconda/lib/python27.zip',
 '/opt/anaconda/lib/python2.7',
 '/opt/anaconda/lib/python2.7/plat-linux2',
 '/opt/anaconda/lib/python2.7/lib-tk',
 '/opt/anaconda/lib/python2.7/lib-old',
 '/opt/anaconda/lib/python2.7/lib-dynload',
 '/opt/anaconda/lib/python2.7/site-packages',
 '/opt/anaconda/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/vagrant/notebooks/cloudera']

# Connect to Cloudera Manager

In [4]:
from cm_api.endpoints.services import ApiService
from cm_api.endpoints.services import ApiServiceSetupInfo
from cm_api.api_client import ApiResource

In [5]:
varFullHost = "pocd-cm581-dev-manager.poc-d.internal"

varAdminUser = "admin"
varAdminPassword = "admin"

varApiResource = ApiResource(varFullHost, version=10, username=varAdminUser, password=varAdminPassword)
varManager = varApiResource.get_cloudera_manager()
varManager

In [7]:
# Get cluster

In [6]:
varClusterName = "POC-D Cluster"


In [7]:
varCluster = varApiResource.get_cluster(varClusterName)


In [8]:
# CDH-5.8.0-1.cdh5.8.0.p0.42-el6.parcel
varParcelVersion = "5.8.0-1.cdh5.8.0.p0.42"

In [11]:
varParcels = [
   { 'name' : "CDH", 'version' : varParcelVersion },
    #{ 'name' : "CDH", 'version' : "5.0.1-1.cdh5.0.1.p0.47" },
    #{ 'name' : "ACCUMULO", 'version' : "1.4.3-cdh4.3.0-beta-3"}
]


In [10]:
def deploy_parcels(cluster, parcels):
   for parcel in parcels:
      p = cluster.get_parcel(parcel['name'], parcel['version'])
      p.start_download()
      while True:
         p = cluster.get_parcel(parcel['name'], parcel['version'])
         if p.stage == "DOWNLOADED":
            break
         if p.state.errors:
            raise Exception(str(p.state.errors))
         print "Downloading %s: %s / %s" % (parcel['name'], p.state.progress, p.state.totalProgress)
         time.sleep(15)
      print "Downloaded %s" % (parcel['name'])
      p.start_distribution()
      while True:
         p = cluster.get_parcel(parcel['name'], parcel['version'])
         if p.stage == "DISTRIBUTED":
            break
         if p.state.errors:
            raise Exception(str(p.state.errors))
         print "Distributing %s: %s / %s" % (parcel['name'], p.state.progress, p.state.totalProgress)
         time.sleep(15)
      print "Distributed %s" % (parcel['name'])
      p.activate()


In [12]:
deploy_parcels(varCluster, varParcels)

Downloaded CDH
Distributing CDH: 1250 / 2500
Distributing CDH: 1250 / 2500
Distributing CDH: 1250 / 2500
Distributing CDH: 1250 / 2500
Distributing CDH: 1250 / 2500
Distributing CDH: 1250 / 2500
Distributing CDH: 1750 / 2500
Distributing CDH: 1750 / 2500
Distributing CDH: 2000 / 2500
Distributed CDH
